In [ ]:
import numpy as np
from pymatgen.core.structure import Structure
from pymatgen.io.vasp.inputs import Poscar
import os

def create_all_nroot7_supercells(poscar_file='POSCAR1.vasp', n=1):
    """
    Create all four types of n√7×n√7 supercells for a given integer n,
    only transforming the x-y plane (keeping z-direction unchanged).
    """
    # Load the original structure
    structure = Structure.from_file(poscar_file)
    
    # Verify the structure is hexagonal
    a, b, c = structure.lattice.abc
    alpha, beta, gamma = structure.lattice.angles
    if not (abs(a - b) < 0.1 and abs(gamma - 120) < 1):
        print("Warning: The input structure doesn't appear to be hexagonal (a ≈ b, γ ≈ 120°)")
    
    # Define all possible √7×√7 transformation matrices (2D only)
    base_matrices_2d = {
        1: np.array([[2, -3], [1, 2]]),    # det = 7
        2: np.array([[2, 3], [-1, 2]]),   # det = 7
        3: np.array([[3, -2], [2, 1]]),   # det = 7
        4: np.array([[1, -2], [3, 2]])    # det = 7
    }
    
    # Create output directory
    output_dir = f'n{n}root7_supercells'
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"\nGenerating n√7×n√7 supercells for n={n} (x-y plane only)...")
    print(f"Original structure: {len(structure)} atoms")
    print(f"Lattice parameters (Å): a={a:.3f}, b={b:.3f}, c={c:.3f}")
    print(f"Lattice angles (°): α={alpha:.1f}, β={beta:.1f}, γ={gamma:.1f}\n")
    
    for matrix_type, base_2d in base_matrices_2d.items():
        # Create 3D transformation matrix (identity for z-direction)
        transformation = np.eye(3)
        transformation[:2,:2] = n * base_2d  # Only modify x-y components
        
        # Create the supercell
        supercell = structure.copy()
        supercell.make_supercell(transformation)
        
        # Calculate scaling factor (should be n√7×n√7 in plane)
        det_2d = np.linalg.det(n * base_2d)
        scaling_factor = np.sqrt(det_2d)
        
        # Write the new POSCAR
        output_file = os.path.join(output_dir, f'POSCAR_n{n}root7_type{matrix_type}')
        poscar = Poscar(supercell)
        poscar.write_file(output_file)
        
        # Print information
        new_a, new_b, new_c = supercell.lattice.abc
        print(f"Type {matrix_type}:")
        print(f"  2D Transformation matrix:\n{base_2d}")
        print(f"  Full 3D transformation:\n{transformation}")
        print(f"  Area scaling: {det_2d:.1f}x (theoretical: {7*n**2:.1f}x)")
        print(f"  New atoms: {len(supercell)}")
        print(f"  New lattice (Å): a={new_a:.3f}, b={new_b:.3f}, c={new_c:.3f}")
        print(f"  New angles (°): γ={supercell.lattice.angles[2]:.1f}")
        print(f"  Output: {output_file}\n")

if __name__ == "__main__":
    # Get user input for n value
    while True:
        try:
            n = 2
            if n > 0:
                break
            print("Please enter a positive integer.")
        except ValueError:
            print("Please enter a valid integer.")
    
    create_all_nroot7_supercells(n=n)